In [4]:
import argparse
import numpy as np
from scipy.sparse import csr_matrix


In [5]:

# Set the random seed for reproducibility
def set_random_seed(seed):
    np.random.seed(seed)

# Load the dataset from the provided path and create a sparse matrix
def load_data_sparse(file_path):
    try:
        # Load raw data
        data = np.load(file_path)
        print(f"Data loaded successfully from {file_path}")
        print(f"Data shape: {data.shape}")

        # Extract user IDs and movie IDs
        user_ids = data[:, 0]
        movie_ids = data[:, 1]

        # Binary representation (1 for rated, 0 for not rated)
        binary_ratings = np.ones(len(user_ids), dtype=np.int8)

        # Create a sparse matrix
        num_users = user_ids.max()
        num_movies = movie_ids.max()
        sparse_matrix = csr_matrix((binary_ratings, (movie_ids - 1, user_ids - 1)), shape=(num_movies, num_users))

        return sparse_matrix
    except Exception as e:
        print(f"Error loading the data or creating sparse matrix: {e}")
        return None
    
def LSH_Imp (sparse_matrix):
    try:
        # Create Permutation Matrix of 
        
        PS = 120 #Permutation Size 
        mov_size = sparse_matrix.shape[0]
        usr_size = sparse_matrix.shape[1]

        #Create the Permutation Matrix with size 100 * Movies Number
        Perm_matrix = np.array([np.random.permutation(mov_size) + 1 for _ in range(PS)]).T
        print(f"Perm_matrix shape: {Perm_matrix.shape}")




        # Initialize the signature matrix with zeros values
        signature_matrix = np.zeros((PS, usr_size))

        # Compute MinHash signatures
        for perm_idx in range(PS):  # Iterate over permutations
            # Current permutation of movie indices
            perm_vector = Perm_matrix[:, perm_idx]  # 1-based indices

            # Convert to 0-based indices
            perm_vector = perm_vector - 1

            # Reorder the sparse matrix rows based on the permutation
            permuted_matrix = sparse_matrix[perm_vector, :]

            # Get the index of the first non zero 
            first_nonzero_indices = np.argmax(permuted_matrix != 0, axis=0)+1

            signature_matrix[perm_idx,] = first_nonzero_indices
        

            print(f"Processed permutation {perm_idx + 1}/{PS}")

        print("MinHash signatures computed.")
        return signature_matrix
    
    except Exception as e:
        print(f"Error loading the data or creating sparse matrix: {e}")
        return None
 




In [6]:
 # Load the user_movie_rating.npy file and convert to a sparse binary matrix
sparse_matrix = load_data_sparse("/Users/amrshata/Desktop/RES/user_movie_rating.npy")
set_random_seed(42)

print(f"MinHash signature matrix shape: {sparse_matrix.shape}")

    # Compute MinHash signatures
minhash_signatures = LSH_Imp(sparse_matrix)

print(f"MinHash signature matrix shape: {minhash_signatures.shape}")



Data loaded successfully from /Users/amrshata/Desktop/RES/user_movie_rating.npy
Data shape: (65225506, 3)
MinHash signature matrix shape: (17770, 103703)
Perm_matrix shape: (17770, 120)
Processed permutation 1/120
Processed permutation 2/120
Processed permutation 3/120
Processed permutation 4/120
Processed permutation 5/120
Processed permutation 6/120
Processed permutation 7/120
Processed permutation 8/120
Processed permutation 9/120
Processed permutation 10/120
Processed permutation 11/120
Processed permutation 12/120
Processed permutation 13/120
Processed permutation 14/120
Processed permutation 15/120
Processed permutation 16/120
Processed permutation 17/120
Processed permutation 18/120
Processed permutation 19/120
Processed permutation 20/120
Processed permutation 21/120
Processed permutation 22/120
Processed permutation 23/120
Processed permutation 24/120
Processed permutation 25/120
Processed permutation 26/120
Processed permutation 27/120
Processed permutation 28/120
Processed p

In [38]:
def apply_banding(signature_matrix, bands=10, rows_per_band=12):

    try:
        num_permutations, num_users = signature_matrix.shape  # Rows = num_permutations, Columns = num_users
        candidate_pairs = set()  # Set to store unique candidate pairs (user1, user2)

        # Iterate over each band
        for band in range(bands):
            start_row = band * rows_per_band  # Start index of the current band
            end_row = start_row + rows_per_band  # End index of the current band

            # Dictionary to store hashed buckets for the current band
            band_hashes = {}

            # Iterate over all users (columns in the signature matrix)
            for user in range(num_users):
                # Extract the mini-signature (rows belonging to this band) for the current user
                band_signature = tuple(signature_matrix[start_row:end_row, user])
                
                # Hash the mini-signature to determine the bucket
                bucket = hash(band_signature)

                # Add the user to the corresponding bucket
                if bucket in band_hashes:
                    # If other users are already in the bucket, add pairs (user, candidate)
                    for candidate in band_hashes[bucket]:
                        # Ensure pairs are ordered as (min, max) to avoid duplicates
                        candidate_pairs.add((min(user, candidate), max(user, candidate)))
                    band_hashes[bucket].append(user)  # Add current user to the bucket
                else:
                    # Create a new bucket with the current user
                    band_hashes[bucket] = [user]

        # Print the total number of candidate pairs found
        print(f"Number of candidate pairs: {len(candidate_pairs)}")
        return candidate_pairs  # Return the set of candidate pairs
    except Exception as e:
        # Handle errors gracefully
        print(f"Error in banding technique: {e}")
        return set()

    

In [39]:
candidate_pairs = apply_banding(minhash_signatures,14,8)


Number of candidate pairs: 255134


In [29]:
# Calculate Jaccard Similarity
def calculate_jaccard_similarity(sparse_matrix, candidate_pairs, threshold=0.5):

    try:
        similar_pairs = []  # List to store pairs with similarity above the threshold


        sparse_matrix_csc = sparse_matrix.tocsc()

        # Iterate over all candidate pairs
        for u1, u2 in candidate_pairs:
            # Retrieve the set of movies rated by user u1 and user u2
            movies_u1 = set(sparse_matrix_csc[:, u1].indices)  # Indices of non-zero entries for user u1
            movies_u2 = set(sparse_matrix_csc[:, u2].indices)  # Indices of non-zero entries for user u2

            # Compute the intersection and union of the movie sets
            intersection = len(movies_u1 & movies_u2)  # Number of common movies rated by both users
            union = len(movies_u1 | movies_u2)         # Total number of unique movies rated by either user

            # Calculate Jaccard Similarity
            jaccard_similarity = intersection / union

            # If similarity exceeds the threshold, add the pair to the result
            if jaccard_similarity > threshold:
                similar_pairs.append((u1 + 1, u2 + 1))  # Convert to 1-based indexing as required

        # Print the total number of similar pairs found
        print(f"Number of similar pairs: {len(similar_pairs)}")
        return similar_pairs  # Return the list of similar pairs
    except Exception as e:
        # Handle errors gracefully and return an empty list
        print(f"Error in Jaccard Similarity calculation: {e}")
        return []


In [40]:
    # Compute Jaccard Similarity for candidate pairs
similar_pairs = calculate_jaccard_similarity(sparse_matrix, candidate_pairs)
similar_pairs

Number of similar pairs: 67


[(52495, 99904),
 (9018, 16793),
 (46192, 47930),
 (77807, 101404),
 (82715, 83057),
 (50547, 101404),
 (27707, 81963),
 (43039, 53267),
 (25621, 49459),
 (14944, 30251),
 (64782, 97494),
 (15039, 18867),
 (8002, 29807),
 (35419, 53267),
 (9018, 75835),
 (9018, 68102),
 (489, 32557),
 (58328, 67675),
 (47930, 100079),
 (51237, 53267),
 (46365, 74191),
 (47930, 63974),
 (72733, 85467),
 (51237, 78973),
 (47930, 77807),
 (58328, 92016),
 (6545, 89023),
 (19153, 58328),
 (81963, 82715),
 (29807, 55105),
 (96810, 100079),
 (47930, 87445),
 (46528, 49457),
 (62898, 71374),
 (1324, 20164),
 (62898, 97494),
 (78517, 91158),
 (8002, 61934),
 (12896, 47930),
 (30525, 55033),
 (33367, 79725),
 (18867, 47930),
 (74191, 81963),
 (7849, 34194),
 (452, 9018),
 (15162, 47930),
 (28512, 58328),
 (48671, 95405),
 (12835, 18867),
 (30251, 72380),
 (47930, 93451),
 (66759, 98995),
 (47930, 91158),
 (35770, 85467),
 (9018, 58328),
 (3584, 58249),
 (9772, 55105),
 (74875, 79321),
 (33756, 75899),
 (74191, 

In [22]:

# Write results to a file
def write_results(output_file, similar_pairs):
    try:
        with open(output_file, 'w') as f:
            for u1, u2 in similar_pairs:
                f.write(f"{u1},{u2}\n")
        print(f"Results written to {output_file}")
    except Exception as e:
        print(f"Error writing results to file: {e}")

In [24]:
    # Write results to the output file
write_results("/Users/amrshata/Desktop/RES/Data-Mining/output_file result.txt", similar_pairs)

Results written to /Users/amrshata/Desktop/RES/Data-Mining/output_file result.txt
